In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevent columns
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode categorical variable
label_encode_gender = LabelEncoder()
data["Gender"] = label_encode_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Onehot encode "Geography"
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combine the Columns
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## save the encoder and scaler
with open('label_encode_gender.pkl', 'wb') as file:
    pickle.dump(label_encode_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [11]:
## Dividing B/W independent and dependent features

X = data.drop('Exited', axis=1)

y = data['Exited']

## spliting the data in train test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scaling the features

scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
X_train.shape

(8000, 12)

In [13]:
y_train.shape

(8000,)

In [14]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
data.shape

(10000, 13)

# ANN Implementation

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
from tensorflow.keras.models import load_model

In [18]:
## Building ANN Model

model = Sequential([
    Dense(64, activation = 'relu', input_shape =(X_train.shape[1],)), ## HL1(Hidden Layer 1) Connected with I/P
    Dense(32, activation = 'relu'), ## HL2
    Dense(1, activation = 'sigmoid') ## Output Layer
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Compiling the model way 1

#### Given below is the code but here the learning rate will be constant so we will use 2nd meathod

### <p style="color: Green;"> model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']) </p>

In [20]:
## Compiling the model way 2
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [21]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [22]:
## Setting up the Tensorboard
## from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
## Also setting up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [33]:
## Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3255 - accuracy: 0.8679 - val_loss: 0.3707 - val_accuracy: 0.8560
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3235 - accuracy: 0.8674 - val_loss: 0.3417 - val_accuracy: 0.8640
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3198 - accuracy: 0.8676 - val_loss: 0.3489 - val_accuracy: 0.8645
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3192 - accuracy: 0.8671 - val_loss: 0.3559 - val_accuracy: 0.8560
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3166 - accuracy: 0.8726 - val_loss: 0.3492 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3131 - accuracy: 0.8715 - val_loss: 0.3510 - val_accuracy: 0.8545
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3107 - accuracy: 0.8723 - val_loss: 0.3563 - val_accuracy: 0.8645

In [34]:
model.save('model.h5')

In [35]:
## Loading TensorBoard extention
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [36]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 12628), started 9:51:31 ago. (Use '!kill 12628' to kill it.)